In [21]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain_openai import ChatOpenAI

import weaviate
from dotenv import load_dotenv,find_dotenv
from weaviate.embedded import EmbeddedOptions
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

import os
import sys


In [22]:
OPENAI_KEY=os.environ["OPENAI_APIKEY"]


In [2]:
sys.path.append('..')

from backend.rag_pipeline import *

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
loader = PyPDFLoader("../data/RaptorContract.pdf")

data = data_loader(loader)

In [4]:
docs = text_splitter(data)

In [5]:
import weaviate

client = weaviate.Client(embedded_options=EmbeddedOptions)


/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


In [6]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [7]:
retriever = WeaviateHybridSearchRetriever(
    client=client,
    index_name="LangChain",
    text_key="text",
    attributes=[],
    create_schema_if_missing=True,
)

In [9]:
retriever.add_documents(docs)

['e88369b3-f9e1-4e10-b74f-0a414edac24d',
 '5c476050-7b64-4674-979d-456a1f8f1ba1',
 'f1819dee-c92a-4673-81ab-0132562241e2',
 '223a8716-93f9-4070-88cc-979cbe014a58',
 '3740da07-6618-459c-8c95-1e95a78d6619',
 '64b45bec-6ba4-4f26-893e-49fbdeb2272a',
 'e97f74b5-7d96-485a-80b2-50aa544efc20',
 '099bd68e-07fb-4d6d-a192-d7b20d41f96e',
 '43076221-4912-4d19-a26c-f61d010eec62',
 '18769e20-7c57-4b3b-9502-0ee80bdfd827',
 'd51ac875-75c3-4ac1-82fc-fe353721f3ed',
 '08bf5f6e-f67f-46bb-aa3e-9f6b10b2e783',
 '5e9b99f3-75b7-4a09-a5b0-bebc3eae869b',
 '48ebcf42-68e6-4348-beac-44127055a7e9',
 'cd3cdb9a-3e75-4b30-b4b2-f69e372810e9',
 '51f08fa4-071e-4f43-889f-0b8860cb2b29',
 '0501b6f6-583a-4570-bf0f-fe72de3e783a',
 '79c7bb5d-0d02-464d-9956-4618cd7869ec',
 '51870439-bbec-473b-a74e-995d5105c953',
 '59b1681d-f0d0-4208-97da-327306ef6a31',
 '11428528-607b-4865-a875-247b863af389',
 '9007fe87-d23f-40d0-93b2-2e0841aa9caf',
 'd161f3a6-9bc4-4d4b-9b29-c63b950daf0d',
 '8ac47264-016b-43ba-8105-927f7464d1f1',
 '37ad371f-e168-

In [10]:
retriever.get_relevant_documents("How much is the Escrow amount?")

[Document(page_content='agreed\nto\nby\nthe\nparties\nand\nthe\ndecision\nof\nthe\nAccounting\nFirm\npursuant\nto\nthis\nSection\n2.07\nis \nreferred\nto\nherein\nas\nthe\n“\nFinal\nClosing\nStatement\n”.\nEach\nof\nthe\nparties\nto\nthis\nAgreement\nagrees \nto\nuse\nits\ncommercially\nreasonable\nefforts\nto\ncooperate\nwith\nthe\nAccounting\nFirm\n(including\nby \nexecuting\na\ncustomary\nengagement\nletter\nreasonably\nacceptable\nto\nit)\nand\nto\ncause\nthe \nAccounting\nFirm\nto\nresolve\nany\nsuch\ndispute\nas\nsoon\nas\npracticable\nafter\nthe\ncommencement\nof \nthe\nAccounting\nFirm’ s\nengagement.\n(e)\nPurchase\nPrice\nAdjustment\n.\nIf\nany\nof\nthe\nClosing\nCash\nAmount,\nthe\nClosing\nDebt \nAmount\nor\nthe\nSeller\nTransaction\nExpenses\n(as\nfinally\ndetermined\npursuant\nto\nthis\nSection\n2.07 \nand\nas\nset\nforth\nin\nthe\nFinal\nClosing\nStatement)\ndiffers\nfrom\nthe\nestimated\namounts\nthereof\nset \nforth\nin\nthe\nEstimated\nClosing\nStatement,\nthe\nPurcha

In [17]:

def create_chians(retriever):
        # Define LLM
    llm = ChatOpenAI(openai_api_key = OPENAI_KEY,model_name="gpt-3.5-turbo", temperature=0)

    # Define prompt template
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)


    retrieval = RunnableParallel(
        {"context": retriever,  "question": RunnablePassthrough()} 
    )

    chain = retrieval | prompt | llm | StrOutputParser()

    return chain

In [18]:
chain = create_chians(retriever)

In [19]:
answer = chain.invoke("how much is the escrow amount?")

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [20]:
answer

'The escrow amount is deposited by the Buyer at Closing and is held in a separate account governed by the terms of the Escrow Agreement. The Escrow Amount will be released to the Company Securityholders in accordance with the provisions of the Escrow Agreement.'